# YOLO to Label Studio  

Purpose - To convert YOLO label files to label studio files.  
Author - Ryan Folks

---

In [1]:
import sys
import os
sys.path.append(os.path.join("..", "..", "..", "ChartExtractor", "src"))

In [18]:
from utilities.annotations import BoundingBox

import json
from pathlib import Path
from typing import Dict, List
from glob import glob
from tqdm.notebook import tqdm

In [46]:
def load_yolo_labels_from_file(filepath: Path) -> str:
    try:
        f = open(str(filepath), 'r')
        labels = f.read()
    except Exception as e:
        print(e)
    finally:
        f.close()
    return labels


def load_bounding_boxes_from_yolo_labels(
    yolo_label_file: str,
    im_width: int,
    im_height: int,
    id_to_category: Dict,
):
    return [
        BoundingBox.from_yolo(label.replace('\n', ''), im_width, im_height, id_to_category)
        for label in yolo_label_file.split("\n")
    ]


def bounding_box_to_ls_result(
    bounding_box: BoundingBox,
    im_width: int,
    im_height: int,
):
    return {
        "to_name": "image",
        "from_name": "label",
        "type": "rectanglelabels",
        "original_width": im_width,
        "original_height": im_height,
        "image_rotation": 0,
        "value" : {
            "x" : (bounding_box.left/im_width)*100,
            "y" : (bounding_box.top/im_height)*100,
            "width" : ((bounding_box.right - bounding_box.left)/im_width)*100,
            "height" : ((bounding_box.bottom - bounding_box.top)/im_height)*100,
            "rotation": 0,
            "rectanglelabels": [
                bounding_box.category
            ]
        }
    }


def ls_task_str_to_dict(ls_task_str: str) -> Dict:
    return json.loads(ls_task_str)


def make_predictions_section(model_version: str):
    return [
        {
            "model_version": model_version,
            "result":[]
        }
    ]

In [94]:
ls_task_str = """
{
  "id": 195,
  "data": {
    "image": "/data/upload/10/f2d492c7-RC_0033_preoperative_postoperative.JPG"
  },
  "annotations": [],
  "predictions": []
}
"""

chart_name = ls_task_str.split(".JPG")[0].split("-")[1]
ylabs = load_yolo_labels_from_file(Path("E:")/"data"/"number_detections"/f"{chart_name}.txt")
ylabs = load_bounding_boxes_from_yolo_labels(ylabs, 4032, 3024, {ix:str(ix) for ix in range(10)})
ylabs = [bounding_box_to_ls_result(bb, 4032, 3024) for bb in ylabs]

ls_task = ls_task_str_to_dict(ls_task_str)
ls_task["predictions"] = make_predictions_section("train_90")
ls_task["predictions"][0]["result"] = ylabs

try:
    f = open(f"E:/data/number_detections_ls/{chart_name}_task.json", 'w')
    f.write(str(ls_task).replace("\'", "\""))
except Exception as e:
    print(e)
finally:
    f.close()